In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/black-friday/train.csv")
test = pd.read_csv("../input/black-friday/test.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

In [ ]:
missing_values = train.isnull().sum().sort_values(ascending = False)
missing_values = missing_values[missing_values > 0]/train.shape[0]
print(f'{missing_values *100} %')

In [ ]:
train['Product_Category_3'].unique()

In [ ]:
train['Product_Category_2'].unique()

In [ ]:
test.isnull().sum()

## **Exploratory Data Analysis**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
train['Age'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(train['Age'])

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(train['City_Category'])

In [ ]:
train['Occupation'].unique()

In [ ]:
train['City_Category'].unique()

In [ ]:
train['Stay_In_Current_City_Years'].unique()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(train['Product_Category_1'],train['Purchase'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(train['Age'],train['Purchase'])

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(train['City_Category'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(train['City_Category'],train['Purchase'])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(train['Stay_In_Current_City_Years'],train['Purchase'])

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(train['Purchase'],color='green')

# Preprocessing 

In [ ]:
def label_encoding(df):
    df['Age']=df['Age'].replace('0-17',17)
    df['Age']=df['Age'].replace('18-25',25)
    df['Age']=df['Age'].replace('26-35',35)
    df['Age']=df['Age'].replace('36-45',45)
    df['Age']=df['Age'].replace('46-50',50)
    df['Age']=df['Age'].replace('51-55',55)
    df['Age']=df['Age'].replace('55+',60)
    df['Gender']=df['Gender'].replace('F',0)
    df['Gender']=df['Gender'].replace('M',1)
    df['City_Category']=df['City_Category'].replace('A',0)
    df['City_Category']=df['City_Category'].replace('B',1)
    df['City_Category']=df['City_Category'].replace('C',2)
    df['Stay_In_Current_City_Years']=df['Stay_In_Current_City_Years'].replace('4+',4)
    return df

In [ ]:
train=label_encoding(train)
test=label_encoding(test)

In [ ]:
train['Stay_In_Current_City_Years']=train['Stay_In_Current_City_Years'].astype(int)
test['Stay_In_Current_City_Years']=test['Stay_In_Current_City_Years'].astype(int)

In [ ]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [ ]:
test.isnull().sum()
train.isnull().sum()

In [ ]:
dfd=train.copy()
dft=test.copy()

In [ ]:
avg_purchase_per_product=pd.DataFrame(train.groupby(['Product_ID'])['Purchase'].mean())
avg_purchase_per_product.reset_index(inplace=True)
avg_purchase_per_user=pd.DataFrame(train.groupby(['User_ID'])['Purchase'].mean())
avg_purchase_per_user.reset_index(inplace=True)

In [ ]:
def create_var(data):
    product_count=pd.DataFrame(data['Product_ID'].value_counts())

    product_count.reset_index(inplace=True)
    product_count=product_count.rename(columns={'index':'Product_ID','Product_ID':'Product_count'})

    data['avg_purchase_per_product']=data['Product_ID'].map(avg_purchase_per_product.set_index('Product_ID')['Purchase'])
    data['product_count']=data['Product_ID'].map(product_count.set_index('Product_ID')['Product_count'])
    data['avg_purchase_per_user']=data['User_ID'].map(avg_purchase_per_user.set_index('User_ID')['Purchase'])

    conditions = [
    (data['Product_Category_1'] != 0) & (data['Product_Category_2'] == 0) & (data['Product_Category_3'] == 0),
    (data['Product_Category_1'] != 0) & (data['Product_Category_2'] != 0) & (data['Product_Category_3'] == 0),
    (data['Product_Category_1'] != 0) & (data['Product_Category_2'] != 0) & (data['Product_Category_3'] != 0)]
    choices = [1, 2, 3]
    data['Category_Count'] = np.select(conditions, choices, default=0)
    
    return data

In [ ]:
train=create_var(train)
test=create_var(test)

In [ ]:
train['Product_ID']=train['Product_ID'].str.slice(2).astype(int)
test['Product_ID']=test['Product_ID'].str.slice(2).astype(int)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test.fillna(0,inplace=True)

In [ ]:
corr=train.corr()
plt.figure(figsize=(20,12))
sns.heatmap(corr,annot=True)

In [ ]:
gender_p=train.groupby(['Gender'])['Purchase'].mean()

In [ ]:
age_p=train.groupby(['Age'])['Purchase'].mean()

In [ ]:
occupation_p=train.groupby(['Occupation'])['Purchase'].mean()

In [ ]:
gender_p

In [ ]:
age_p

In [ ]:
occupation_p

In [ ]:
print(occupation_p.mean())
print(occupation_p.std())

In [ ]:
city_cat_p=train.groupby(['City_Category'])['Purchase'].mean()

In [ ]:
city_cat_p

In [ ]:
marital_p=train.groupby(['Marital_Status'])['Purchase'].mean()

In [ ]:
marital_p

In [ ]:
years_p=train.groupby(['Stay_In_Current_City_Years'])['Purchase'].mean()

In [ ]:
years_p

# Model Building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost.sklearn import XGBRegressor

In [ ]:
X=train.drop('Purchase',axis=1)

In [ ]:
y=train['Purchase']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

## Linear Regression

In [ ]:
reg=linear_model.LinearRegression()

In [ ]:
lm_model=reg.fit(X_train,y_train)
pred=lm_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,pred))

## Random Forest

In [ ]:
RF_reg=RandomForestRegressor()

In [ ]:
RF_model=RF_reg.fit(X_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(y_test,pred))

## XGBoost Regressor

In [ ]:
xgb=XGBRegressor()

In [ ]:
XGB_model=xgb.fit(X_train,y_train)

In [ ]:
xg_pred=XGB_model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,xg_pred))